# 

In [1]:
import pandas as pd 
import numpy as np
pd.set_option('display.max_columns', None)

pd.set_option('display.max_rows', None)

In [2]:
generation = pd.read_csv('generation.csv', sep=';')
temperature = pd.read_csv('temperature.csv',sep=';')
sample_submission = pd.read_csv('sample_submission.csv')
## dropping empty rows from datasets
empty_rows_gener = generation[generation.isnull().any(axis = 1)].index
generation.drop(empty_rows_gener,inplace= True)
L = temperature.index[temperature.isna().all(axis=1)].tolist()
temperature.drop(L,inplace= True)
## temperature kısmında generation olmayan kısım tet
main = pd.merge(temperature,generation,on='DateTime',how = 'left')
main['DateTime']= pd.to_datetime(main['DateTime'])
main['AirTemperature'] = main['AirTemperature'].str.replace(',', '.').astype(float)
main['EffectiveCloudCover'] = main['EffectiveCloudCover'].str.replace(',', '.').astype(float)
main['ComfortTemperature'] = main['ComfortTemperature'].str.replace(',', '.').astype(float)
main['RelativeHumidity'] = main['RelativeHumidity'].str.replace(',', '.').astype(float)
main['WindSpeed'] = main['WindSpeed'].str.replace(',', '.').astype(float)
main['Generation'] = main['Generation'].str.replace(',', '.').astype(float)
#main = main.drop(['WWCode'],axis = 1)
indexer = pd.api.indexers.FixedForwardWindowIndexer(window_size=2)
main['two_hour_airtem'] = main.AirTemperature.rolling(window=indexer, min_periods=1).mean()
main['two_hour_comfort'] = main.ComfortTemperature.rolling(window=indexer, min_periods=1).mean()
main['two_hour_cloud'] = main.EffectiveCloudCover.rolling(window=indexer, min_periods=1).mean()
main['two_hour_hum'] = main.RelativeHumidity.rolling(window=indexer, min_periods=1).mean()
main['two_hour_wind'] = main.WindSpeed.rolling(window=indexer, min_periods=1).mean()


In [4]:
## Addind two hourly mean atrributes of humidity, cloudcover and windspeed
## lag features
main.two_hour_hum = main.two_hour_hum + 0.0001
main.two_hour_cloud = main.two_hour_cloud + 0.0001
main.two_hour_wind = main.two_hour_wind + 0.0001

main['prop_hum'] = main.two_hour_hum / main.two_hour_hum.shift(1)
main['prop_cloud'] = main.two_hour_cloud / main.two_hour_cloud.shift(1)
main['prop_wind'] = main.two_hour_wind / main.two_hour_wind.shift(1)

In [5]:
## Addind three hourly mean atrributes of humidity, cloudcover and windspeed
## lead features

main['prop_hum2'] = main.two_hour_hum / main.two_hour_hum.shift(-3)
main['prop_cloud2'] = main.two_hour_cloud / main.two_hour_cloud.shift(-3)
main['prop_wind2'] = main.two_hour_wind / main.two_hour_wind.shift(-3)

In [6]:
## Addind two hourly mean atrributes of humidity, cloudcover and windspeed

main['prop_hum3'] = main.two_hour_hum / main.two_hour_hum.shift(-2)
main['prop_cloud3'] = main.two_hour_cloud / main.two_hour_cloud.shift(-2)
main['prop_wind3'] = main.two_hour_wind / main.two_hour_wind.shift(-2)

In [7]:
## Addind two hourly mean atrributes of humidity, cloudcover and windspeed

main['prop_hum4'] = main.two_hour_hum / main.two_hour_hum.shift(-1)
main['prop_cloud4'] = main.two_hour_cloud / main.two_hour_cloud.shift(-1)
main['prop_wind4'] = main.two_hour_wind / main.two_hour_wind.shift(-1)

# CALCULATION OF SUN POSITION

In [8]:
import math
def sunpos(when, location, refraction):
# Extract the passed data
    year, month, day, hour, minute, second, timezone = when
    latitude, longitude = location
# Math typing shortcuts
    rad, deg = math.radians, math.degrees
    sin, cos, tan = math.sin, math.cos, math.tan
    asin, atan2 = math.asin, math.atan2
# Convert latitude and longitude to radians
    rlat = rad(latitude)
    rlon = rad(longitude)
# Decimal hour of the day at Greenwich
    greenwichtime = hour - timezone + minute / 60 + second / 3600
# Days from J2000, accurate from 1901 to 2099
    daynum = (
        367 * year
        - 7 * (year + (month + 9) // 12) // 4
        + 275 * month // 9
        + day
        - 730531.5
        + greenwichtime / 24
    )
# Mean longitude of the sun
    mean_long = daynum * 0.01720279239 + 4.894967873
# Mean anomaly of the Sun
    mean_anom = daynum * 0.01720197034 + 6.240040768
# Ecliptic longitude of the sun
    eclip_long = (
        mean_long
        + 0.03342305518 * sin(mean_anom)
        + 0.0003490658504 * sin(2 * mean_anom)
    )
# Obliquity of the ecliptic
    obliquity = 0.4090877234 - 0.000000006981317008 * daynum
# Right ascension of the sun
    rasc = atan2(cos(obliquity) * sin(eclip_long), cos(eclip_long))
# Declination of the sun
    decl = asin(sin(obliquity) * sin(eclip_long))
# Local sidereal time
    sidereal = 4.894961213 + 6.300388099 * daynum + rlon
# Hour angle of the sun
    hour_ang = sidereal - rasc
# Local elevation of the sun
    elevation = asin(sin(decl) * sin(rlat) + cos(decl) * cos(rlat) * cos(hour_ang))
    azimuth = atan2(
        -cos(decl) * cos(rlat) * sin(hour_ang),
        sin(decl) - sin(rlat) * sin(elevation),
    )
    azimuth = into_range(deg(azimuth), 0, 360)
    elevation = into_range(deg(elevation), -180, 180)
    if refraction:
        targ = rad((elevation + (10.3 / (elevation + 5.11))))
        elevation += (1.02 / tan(targ)) / 60
    return (round(azimuth, 2), round(elevation, 2))
def into_range(x, range_min, range_max):
    shiftedx = x - range_min
    delta = range_max - range_min
    return (((shiftedx % delta) + delta) % delta) + range_min
if __name__ == "__main__":
    location = (40.602778, -104.741667)
    when = (2022, 7, 4, 11, 20, 0, -6)
    azimuth, elevation = sunpos(when, location, True)
def get_angles(dt, refraction):
    year, month, day = dt.year, dt.month, dt.day
    hour, minute, second = dt.hour, dt.minute, dt.second
    timezone = 3
    when = (year, month, day, hour, minute, second, timezone)
    location = (39.925533, 32.866287)
    
    angles = sunpos(when, location, refraction)
    return angles
def get_azimuth(dt, refraction=False):
    angles = get_angles(dt, refraction)
    return angles[0]
def get_elevation(dt, refraction=False):
    angles = get_angles(dt, refraction)
    return angles[1]
def abs_sin(value):
    return abs(np.sin(value))
main['Azimuth'] = main.DateTime.apply(get_azimuth)
main['Elevation'] = main.DateTime.apply(get_elevation)
indexer = pd.api.indexers.FixedForwardWindowIndexer(window_size=2)
main['mean_ele'] = main.Elevation.rolling(window=indexer, min_periods=1).mean()

In [9]:
## Adding some additional features (square of these two features)
main['sq_cloud'] = main.two_hour_cloud ** 2
main['sq_hum'] = main.two_hour_hum ** 2

In [10]:
main = main[['DateTime','Generation','Elevation','Azimuth','two_hour_hum','two_hour_cloud','mean_ele',
             'prop_hum','prop_cloud',
             'prop_hum2','prop_cloud2',
             'prop_cloud3','prop_hum3',
             'prop_cloud4','prop_hum4',
             'WWCode','WindDirection',
             'sq_cloud','sq_hum','WindSpeed'
             
            ]]

In [11]:
## One hot encoding of humidity feature

from sklearn.cluster import KMeans
import numpy as np
from sklearn.preprocessing import MinMaxScaler
X= main[['two_hour_hum']]
sc = MinMaxScaler()
X = sc.fit_transform(X)
kmeans = KMeans(n_clusters=5, random_state=0).fit(X)
b = kmeans.labels_

a = kmeans.predict(X)

c = kmeans.cluster_centers_
main['cluster_c'] = b

main['hh1'] = np.where((main.cluster_c == 0), 1, 0)
main['hh2'] = np.where((main.cluster_c == 1), 1, 0)
main['hh3'] = np.where((main.cluster_c == 2), 1, 0)
main['hh4'] = np.where((main.cluster_c == 3), 1, 0)
main['hh5'] = np.where((main.cluster_c == 4), 1, 0)
main['hh6'] = np.where((main.cluster_c == 5), 1, 0)
#main['ws7'] = np.where((main.cluster_c == 6), 1, 0)
#main['ws8'] = np.where((main.cluster_c == 7), 1, 0)
#main['ws9'] = np.where((main.cluster_c == 8), 1, 0)
#main['ws10'] = np.where((main.cluster_c == 9), 1, 0)

main.drop(['cluster_c','two_hour_hum'],axis = 1,inplace = True)


In [12]:
## One hot encoding of WindSpeed feature

from sklearn.cluster import KMeans
import numpy as np
from sklearn.preprocessing import MinMaxScaler
X= main[['WindSpeed']]
sc = MinMaxScaler()
X = sc.fit_transform(X)
kmeans = KMeans(n_clusters=7, random_state=0).fit(X)
b = kmeans.labels_

a = kmeans.predict(X)

c = kmeans.cluster_centers_
main['cluster_c'] = b

main['ws1'] = np.where((main.cluster_c == 0), 1, 0)
main['ws2'] = np.where((main.cluster_c == 1), 1, 0)
main['ws3'] = np.where((main.cluster_c == 2), 1, 0)
main['ws4'] = np.where((main.cluster_c == 3), 1, 0)
main['ws5'] = np.where((main.cluster_c == 4), 1, 0)
main['ws6'] = np.where((main.cluster_c == 5), 1, 0)
main['ws7'] = np.where((main.cluster_c == 6), 1, 0)
#main['ws8'] = np.where((main.cluster_c == 7), 1, 0)
#main['ws9'] = np.where((main.cluster_c == 8), 1, 0)
#main['ws10'] = np.where((main.cluster_c == 9), 1, 0)

main.drop(['cluster_c','WindSpeed'],axis = 1,inplace = True)


In [13]:
## One hot encoding of WindDirection feature

from sklearn.cluster import KMeans
import numpy as np
from sklearn.preprocessing import MinMaxScaler
X= main[['WindDirection']]
sc = MinMaxScaler()
X = sc.fit_transform(X)
kmeans = KMeans(n_clusters=10, random_state=0).fit(X)
b = kmeans.labels_

a = kmeans.predict(X)

c = kmeans.cluster_centers_
main['cluster_c'] = b

main['wd1'] = np.where((main.cluster_c == 0), 1, 0)
main['wd2'] = np.where((main.cluster_c == 1), 1, 0)
main['wd3'] = np.where((main.cluster_c == 2), 1, 0)
main['wd4'] = np.where((main.cluster_c == 3), 1, 0)
main['wd5'] = np.where((main.cluster_c == 4), 1, 0)
main['wd6'] = np.where((main.cluster_c == 5), 1, 0)
main['wd7'] = np.where((main.cluster_c == 6), 1, 0)
main['wd8'] = np.where((main.cluster_c == 7), 1, 0)
main['wd9'] = np.where((main.cluster_c == 8), 1, 0)
main['wd10'] = np.where((main.cluster_c == 9), 1, 0)

main.drop(['cluster_c','WindDirection'],axis = 1,inplace = True)


In [14]:
index_names = main[main['WWCode'].isna()].index
main.drop(index_names, inplace = True)


# K-means Clustering for some features

In [16]:
from sklearn.cluster import KMeans
import numpy as np
from sklearn.preprocessing import MinMaxScaler
X= main[['WWCode']]
sc = MinMaxScaler()
X = sc.fit_transform(X)
kmeans = KMeans(n_clusters=4, random_state=0).fit(X)
b = kmeans.labels_

a = kmeans.predict(X)

c = kmeans.cluster_centers_
main['cluster'] = b

main['w1'] = np.where((main.cluster == 1), 1, 0)
main['w2'] = np.where((main.cluster == 2), 1, 0)
main['w3'] = np.where((main.cluster == 3), 1, 0)
main['w4'] = np.where((main.cluster == 0), 1, 0)
#main['w5'] = np.where((main.cluster == 4), 1, 0)
#main['w6'] = np.where((main.cluster == 5), 1, 0)
#main['w7'] = np.where((main.cluster == 6), 1, 0)

main.drop(['cluster','WWCode'],axis = 1, inplace=True)

In [17]:
main.loc[main.mean_ele < 0, 'Azimuth'] = 0
main.loc[main.mean_ele < 0, 'mean_ele'] = 0

main['new_azi'] = 180 - main.Azimuth
main.drop('Azimuth',axis= 1,inplace=True)

In [19]:
from sklearn.cluster import KMeans
import numpy as np
from sklearn.preprocessing import MinMaxScaler
X= main[['mean_ele']]
sc = MinMaxScaler()
X = sc.fit_transform(X)
kmeans = KMeans(n_clusters=10, random_state=0).fit(X)
b = kmeans.labels_

a = kmeans.predict(X)

c = kmeans.cluster_centers_
main['cluster'] = b


main['cluster1'] = np.where((main.cluster == 1), 1, 0)
main['cluster2'] = np.where((main.cluster == 2), 1, 0)
main['cluster3'] = np.where((main.cluster == 3), 1, 0)
main['cluster4'] = np.where((main.cluster == 4), 1, 0)
main['cluster5'] = np.where((main.cluster == 5), 1, 0)
main['cluster6'] = np.where((main.cluster == 6), 1, 0)
main['cluster7'] = np.where((main.cluster == 7), 1, 0)
main['cluster8'] = np.where((main.cluster == 8), 1, 0)
main['cluster9'] = np.where((main.cluster == 9), 1, 0)
main['cluster10'] = np.where((main.cluster == 0), 1, 0)


main.drop(['cluster'],axis = 1,inplace = True)
main.drop(['Elevation'],axis = 1,inplace = True)
main.drop(['mean_ele'],axis = 1,inplace = True)

# Handling missing variables

In [22]:
test_set.at[26303,'prop_hum2'] = test_set.at[26300,'prop_hum2']
test_set.at[26302,'prop_hum2'] = test_set.at[26300,'prop_hum2']
test_set.at[26301,'prop_hum2'] = test_set.at[26300,'prop_hum2']

test_set.at[26303,'prop_cloud2'] = test_set.at[26300,'prop_cloud2']
test_set.at[26302,'prop_cloud2'] = test_set.at[26300,'prop_cloud2']
test_set.at[26301,'prop_cloud2'] = test_set.at[26300,'prop_cloud2']

test_set.at[26303,'prop_cloud3'] = test_set.at[26301,'prop_cloud3']
test_set.at[26302,'prop_cloud3'] = test_set.at[26301,'prop_cloud3']

test_set.at[26303,'prop_hum3'] = test_set.at[26301,'prop_hum3']
test_set.at[26302,'prop_hum3'] = test_set.at[26301,'prop_hum3']

test_set.at[26303,'prop_cloud4'] = test_set.at[26302,'prop_cloud4']

test_set.at[26303,'prop_hum4'] = test_set.at[26302,'prop_hum4']

In [24]:
main = main[main.Generation.isna()==False]

In [25]:
main = main.dropna()

In [26]:
main = main[main.DateTime.dt.year != 2019]

# Train test split

In [166]:
train = main[((main.DateTime.dt.month != 11) & (main.DateTime.dt.year == 2021)) | (main.DateTime.dt.year == 2020)]
test = main[(main.DateTime.dt.month == 11) & (main.DateTime.dt.year == 2021)]
train = train.dropna()
test = test.dropna()

In [27]:
X_train = main.drop(['Generation','DateTime'],axis = 1)
y_train = main['Generation']
X_test = test_set.drop(['Generation','DateTime'],axis = 1)
y_test = test_set['Generation']

In [168]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
model = RandomForestRegressor(max_depth= 100, max_features= 'auto', min_samples_leaf= 20, 
                              min_samples_split= 2,n_estimators= 50).fit(X_train, y_train)

predictions = model.predict(X_test)
from sklearn.metrics import mean_squared_error
mse = mean_squared_error(predictions,y_test)
rmse = mse**0.5
rmse

20.25863254035309

In [169]:
import lightgbm as lgbm
model = lgbm.sklearn.LGBMRegressor()
model.fit(X_train,y_train)
predictions = model.predict(X_test)
from sklearn.metrics import mean_squared_error
mse = mean_squared_error(predictions,y_test)
rmse = mse**0.5
rmse

15.842843232867692

In [28]:
from catboost import CatBoostRegressor
model = CatBoostRegressor().fit(X_train,y_train,verbose = False)
predictions = model.predict(X_test)


In [171]:
from sklearn.metrics import mean_squared_error
mse = mean_squared_error(predictions,y_test)
rmse = mse**0.5
rmse

14.22125224028979

In [173]:
from sklearn.ensemble import RandomForestRegressor

from sklearn.datasets import make_friedman1
from sklearn.feature_selection import RFE
from sklearn.svm import SVR
estimator = RandomForestRegressor(max_depth= 100, max_features= 'auto', min_samples_leaf= 20, 
                              min_samples_split= 2,n_estimators= 50)
selector = RFE(estimator, n_features_to_select=45, step=1)
selector = selector.fit(X_train, y_train)


In [29]:
from numpy.random import randn
from numpy.random import seed
from scipy.stats import pearsonr

corr, _ = pearsonr(train.prop_cloud,train.Generation)
print('Pearsons correlation: %.3f' % corr)

Pearsons correlation: 0.028


In [30]:
X_test['pred'] = predictions
X_test['real'] = test.Generation
X_test['date'] = test.DateTime
X_test['diff'] = X_test.real- X_test.pred
X_test['hour'] = X_test.date.dt.hour

# Submission steps

In [31]:
sample_submission['sada'] = predictions


In [32]:
sample_submission.drop(['Generation'],axis= 1,inplace = True)


In [33]:
sample_submission.rename(columns = {'sada':'Generation'},inplace = True)


In [34]:
sample_submission = sample_submission.set_index('DateTime')


In [35]:
sample_submission.to_csv('enerjisa_sub15.csv',sep=',')